In [1]:
!pip install ipywidgets matplotlib numpy imageio imageio-ffmpeg tqdm -q

In [19]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import numpy as np
import os
import imageio.v2 as imageio
from IPython.display import Video
from concurrent.futures import ThreadPoolExecutor
import time
from tqdm import tqdm
from PIL import Image
from IPython.display import HTML

In [3]:
# Defining the Mandelbrot set class with the extended functionality from the third script
class MandelbrotSet:
    def __init__(self, max_iterations: int, escape_radius: float = 2.0):
        self.max_iterations = max_iterations
        self.escape_radius = escape_radius

    def __contains__(self, c: complex) -> bool:
        return self.stability(c) == 1

    def stability(self, c: complex, smooth=False, clamp=True) -> float:
        value = self.escape_count(c, smooth) / self.max_iterations
        return max(0.0, min(value, 1.0)) if clamp else value

    def escape_count(self, c: complex, smooth=False) -> float:
        z = 0
        for iteration in range(self.max_iterations):
            z = z**2 + c
            if abs(z) > self.escape_radius:
                if smooth:
                    return iteration + 1 - np.log(np.log(abs(z))) / np.log(2)
                return iteration
        return self.max_iterations

# Function to generate and plot the Mandelbrot set
def plot_mandelbrot(max_iterations, escape_radius, smooth, width=800, height=600, x_center=-0.5, y_center=0, zoom=1):
    x_min, x_max = x_center - 1.5/(zoom*2), x_center + 1.5/(zoom*2)
    y_min, y_max = y_center - 1/(zoom*2), y_center + 1/(zoom*2)
    x, y = np.linspace(x_min, x_max, width), np.linspace(y_min, y_max, height)
    X, Y = np.meshgrid(x, y)
    C = X + 1j*Y
    mandelbrot_set = MandelbrotSet(max_iterations, escape_radius)
    output = np.zeros(C.shape, dtype=float)
    for i in range(height):
        for j in range(width):
            output[i, j] = mandelbrot_set.stability(C[i, j], smooth=smooth)

    plt.figure(figsize=(10, 8))
    plt.imshow(output, extent=[x_min, x_max, y_min, y_max], cmap='hot')
    plt.colorbar()
    plt.title("Mandelbrot Set")
    plt.xlabel("Re")
    plt.ylabel("Im")
    plt.show()

# Creating interactive widgets
max_iterations_slider = widgets.IntSlider(value=100, min=10, max=1000, step=10, description='Max Iterations:')
escape_radius_slider = widgets.FloatSlider(value=2.0, min=1.0, max=4.0, step=0.1, description='Escape Radius:')
smooth_checkbox = widgets.Checkbox(value=False, description='Smooth Coloring')

# Button to generate the plot
plot_button = widgets.Button(description="Plot Mandelbrot Set")

# Output widget to display the plot
output_widget = widgets.Output()

def on_plot_button_clicked(b):
    with output_widget:
        clear_output(wait=True)
        plot_mandelbrot(max_iterations_slider.value, escape_radius_slider.value, smooth_checkbox.value)

plot_button.on_click(on_plot_button_clicked)

# Displaying the widgets
widgets.VBox([max_iterations_slider, escape_radius_slider, smooth_checkbox, plot_button, output_widget])


In [28]:
# Ensure the output directory exists
output_dir = 'viu_zoom'
os.makedirs(output_dir, exist_ok=True)


def plot_zoomed_mandelbrot(center, zoom, frame_number, base_iterations=100):
    # Dynamic adjustment of max_iterations based on zoom level
    max_iterations = int(base_iterations * np.log2(zoom + 1))
    # Ensure the figure size and DPI lead to image dimensions divisible by 16
    dpi = 400  # High resolution

    # Target dimensions in pixels, divisible by 16
    target_width_px = 1920  # Example width, adjust as needed
    target_height_px = 1080  # Example height, adjust as needed

    # Ensure dimensions are divisible by 16
    target_width_px -= target_width_px % 16
    target_height_px -= target_height_px % 16

    # Calculate figure size in inches
    fig_width = target_width_px / dpi
    fig_height = target_height_px / dpi

    aspect_ratio = fig_width / fig_height
    scale = 2.0 / zoom
    xmin, xmax = center[0] - scale * aspect_ratio / 2, center[0] + scale * aspect_ratio / 2
    ymin, ymax = center[1] - scale / 2, center[1] + scale / 2

    # Generate the Mandelbrot set within the viewport
    mandelbrot_set = MandelbrotSet(max_iterations)
    x = np.linspace(xmin, xmax, target_width_px)
    y = np.linspace(ymin, ymax, target_height_px)
    X, Y = np.meshgrid(x, y)
    C = X + 1j*Y
    output = np.array([[mandelbrot_set.stability(c, smooth=True) for c in row] for row in C])

    # Plotting
    plt.figure(figsize=(fig_width, fig_height), dpi=dpi)
    plt.imshow(output, extent=[xmin, xmax, ymin, ymax], cmap='hot', interpolation='bilinear')
    plt.axis('off')
    plt.savefig(f'{output_dir}/frame_{frame_number:04d}.png', bbox_inches='tight', pad_inches=0)
    plt.close()

# Initial parameters
center = (-0.74.5, -0.19)  # Center point for zooming
zoom = 1  # Initial zoom level
frames = 30  # Number of frames in the animation
# Output video file
video_file = 'mandelbrot_zoom.mp4'
writer = imageio.get_writer(video_file, fps= frames / 10)
def task(n):
    print(f"Processing {n}")
    time.sleep(1)  # Simulate a time-consuming task
    return n * n

# Number of workers equals the number of threads you wish to utilize
number_of_workers = 8
# Create a ThreadPoolExecutor
with ThreadPoolExecutor(max_workers=number_of_workers) as executor:
    for frame in tqdm(range(frames), desc="Generating frames"):
        plot_zoomed_mandelbrot(center, zoom, frame)
        zoom *= 1.6  # Increase the zoom level for the next frame
    for frame_number in range(frames):
        frame_path = f'{output_dir}/frame_{frame_number:04d}.png'
        frame = Image.open(frame_path)
    
    # Resize the frame to make sure dimensions are divisible by 16
        original_width, original_height = frame.size
        new_width = original_width - (original_width % 16)
        new_height = original_height - (original_height % 16)
    
        # If the original dimensions are already correct, no resizing is done
        if (new_width != original_width) or (new_height != original_height):
            frame = frame.resize((new_width, new_height), Image.Resampling.LANCZOS)  # Updated to use Resampling.LANCZOS
    
        # Convert the PIL image to a numpy array so it can be written by imageio
        frame_np = np.array(frame)
        writer.append_data(frame_np)

# Close the writer to finish writing the video file
writer.close()

# Set desired display dimensions
display_width = 800  # Adjust width as needed
display_height = 600  # Adjust height as needed, maintain aspect ratio

# Embed video with custom size
HTML(f"""
<video width="{display_width}" height="{display_height}" controls>
  <source src="{video_file}" type="video/mp4">
</video>
""")
# Display the video in the Jupyter notebook

Generating frames: 100%|████████████████████████████████████████████████████████████████████| 30/30 [06:57<00:00, 13.93s/it]
[rawvideo @ 0x12cf05f60] Stream #0: not enough frames to estimate rate; consider increasing probesize
